# Projet numérique


# Pas fixe :

In [ ]:
def solve_euler_explicite(f,x0,dt,t0,tf):
    T=[t0]
    X=[x0]
    t=t0
    x=x0
    n = int((tf-t0)/dt) # nombre de pas à faire entre t0 et tf
    for i in range(n) :
        dx = f(x,t)*dt
        x += dx
        t += dt
        T.append(t)
        X.append(x)
    return T,X

La fonction $solve\_euler\_explicite$ utilise la méthode numérique d'Euler pour obtenir une solution approchée de la solution de l'équation différentielle $x'(t)=f(x,t)$, telle que $x(t_0)=x_0$. C'est une méthode d'ordre 1 : l'erreur commise sur un intervalle de taille $(t_f-t_0)$ est en $O(dt)$.

In [ ]:
def solve_heun(f,x0,dt,t0,tf):
    T=[t0]
    X=[x0]
    t=t0
    x=x0
    n = int((tf-t0)/dt)
    for i in range(n) :
        dx = dt/2*(f(x,t)+f(x+dt*f(x,t),t+dt)) # on utilise la méthode de Heun pour obtenir une approximation de x(j+1) à l'ordre 2
        x += dx
        t += dt
        T.append(t)
        X.append(x)
    return T,X

La méthode de Heun utilisée dans la fonction précédente est d'ordre 2 : l'erreur est en $O(dt^2)$. Cet algorithme est donc bien plus efficace, car il limite grandement l'erreur sur la solution, sans trop augmenter les temps de calcul.

On peut comparer ces deux algorithmes en résolvant une équation différentielle dont on connait une solution exacte, et en utilisant différents pas de temps.
On prend l'exemple de $f(x,t)=1/x$, de solution $g(t)=\sqrt{1+2t}$.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

f = lambda x,t : 1/x # fonction telle que dx/dt = f(x,t)

g = lambda t : np.sqrt(1+2*t) # solution g de l'équation différentielle

def test_ecart(f,g,x0,dt,t0,tf): # renvoie les écarts maximaux entre la solution exacte g et les solutions approchées par Euler et Heun
    T,X1=solve_euler_explicite(f,x0,dt,t0,tf)
    T,X2=solve_heun(f,x0,dt,t0,tf)
    Y = [g(t) for t in T]
    L_ecart_euler = np.abs(np.array(Y)-np.array(X1))
    L_ecart_heun = np.abs(np.array(Y)-np.array(X2))
    ecart_euler = np.max(L_ecart_euler)
    ecart_heun = np.max(L_ecart_heun)
    return ecart_euler,ecart_heun

x0=1
t0=0
tf = 10

L_dt = [1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.1,0.05,0.025]
Euler = []
Heun = []
for dt in L_dt :
    e1,e2 = test_ecart(f,g,x0,dt,t0,tf)
    Euler.append(e1)
    Heun.append(e2)

plt.plot(L_dt,Euler,'r',label='Euler')
plt.plot(L_dt,Heun,'g',label='Heun')
plt.legend()
plt.xlabel('dt')
plt.ylabel('erreur')
plt.show()

In [ ]:
T,X1=solve_euler_explicite(f,x0,0.5,t0,tf)
T,X2=solve_heun(f,x0,0.5,t0,tf)
Y = [g(t) for t in T]
plt.plot(T,X1,'r',label='Euler')
plt.plot(T,X2,'g',label='Heun')
plt.plot(T,Y,'b',label='solution exacte')
plt.legend()
plt.xlabel('t')
plt.show()

On voit que pour $dt$ suffisament petit, les erreurs commises par les méthodes d'Euler et de Heun tendent vers 0. Cependant, on observe que l'erreur croit beaucoup plus rapidement pour la méthode d'Euler que pour la méthode de Heun. Ainsi, pour un pas de temps $dt = 0.5$, la solution exacte et la solution approchée par Heun sont presque confondues, alors qu'on observe un écart non-négligeable avec la solution approchée d'Euler.


Les méthodes présentées ci-dessus utilisent un pas de temps constant. Cependant certains systèmes ne peuvent être résolus avec un pas fixe (comme par exemple les systèmes dits raides). Pour remédier à ce problème il est possible d'utiliser des pas variables qui vont permettre d'adapter le pas de temps au cours de la résolution.

# Pas variable


	
On commence par s'interesser à l'erreur absolue locale que l'on souhaite estimer. Cette estimation de l'erreur commise permet de choisir un pas de temps plus adapté (en diminuant le pas de temps si l'erreur est trop élevée ou en l'augmentant si elle est bien au dessous de la tolérance voulue). On cherche dans un premier temps à exprimer $e^{j+1}$ qui représente cette erreur.
	
$$e^{j+1}=x(t_j)+\int_{t_j}^{t_{j+1}} f(s,x(s))ds-x^{j+1}$$


or $$f(s,x(s))=x'(s)$$ 
donc $$\int_{t_j}^{t_{j+1}} f(s,x(s))ds=x(t_{j+1})-x(t_j)$$
et $$e^{j+1}= x(t_{j+1})-x^{j+1}$$

On suppose que l'on a $f$ $C^1$ donc $x$ est $C^2$, en faisant un développement limité au second ordre on obtient :

$$x(t_{j+1})=x(t_j)+ \Delta t f(t_j,x(t_j))+\frac{\Delta t^2}{2} [\partial_t{f(t_j,x(t_j))}+\partial_x{f(t_j,x(t_j))}f(t_j,x(t_j))]+O(\Delta t^3)$$

et de même en faisant un développement limité de f :

$$f(x_{j+1},x(t_{j+1}))=f(t_j,x(t_j))+\Delta t [\partial_t{f(t_j,x(t_j)}+\partial_x{f(t_j,x(t_j))}f(t_j,x(t_j))] + O(\Delta t^2)$$

Développons l'expression de $e^{j+1}$ en se servant du fait que l'on utilise une méthode d'Euler explicite.

Remarque : On notera indifférement $x^j$ et $x(t_j)$ qui représentent ici la même chose.

$$\begin{eqnarray*}
e^{j+1} & = & x(t_{j+1})-x^{j+1}  \\
& = & x(t_j)+ \Delta t f(t_j,x(t_j))+\frac{\Delta t^2}{2} (\partial_t{f(t_j,x(t_j)}+\partial_x{f(t_j,x(t_j)}f(t_j,x(t_j)) \\
&  & -(x^j + \Delta t f(t_j,x(t_j))) + O(\Delta t^3) \\
&=& \frac{\Delta t^2}{2} [\partial_t{f(t_j,x(t_j)}+\partial_x{f(t_j,x(t_j))}f(t_j,x(t_j))] + O(\Delta t^3) \\
\end{eqnarray*}$$



D'un autre côté on a :

$$\begin{eqnarray*}
f(t_{j+1},x^{j+1}) & = & f(t_j+\Delta t,x^j+\Delta t f(t_j,x^j) )  \\ 
& = & f(t_j,x^j) + \Delta t [\partial_t{f(t_j,x(t_j))}+\partial_x{f(t_j,x(t_j))}f(t_j,x(t_j))] + O(\Delta t^2) \\	
\end{eqnarray*}$$

donc :
$$\frac{\Delta t}{2}[f(t_{j+1},x^{j+1})-f(t_j,x^j)] = \frac{\Delta t^2}{2} [\partial_t{f(t_j,x(t_j))}+\partial_x{f(t_j,x(t_j))}f(t_j,x(t_j))] + O(\Delta t^3) $$
On déduit donc que :
$$e^{j+1} = \frac{\Delta t}{2}[f(x_{j+1},x(t_{j+1}))-f(t_j,x^j)] + O(\Delta t^3) $$
finalement :
$$||e^{j+1}||=\frac{\Delta t}{2}||f(t_{j+1},x^{j+1})-f(t_j,x^j)|| + O(\Delta t^3) $$
	
	
	
	
Notre but est de pouvoir adapter le pas de temps tout en acceptant une tolérance $Tol_{abs}$ sur l'erreur.
Au cours de notre réflexion nous avons vu deux stratégies d'adaptation possibles : 
	
Dans la première stratégie, nous estimons le pas de temps $j+2$ en se basant sur le pas de temps $j+1$. Pour cela nous corrigeons le pas en $j+1$ grâce à une évaluation de l'erreur commise en $j+1$. Cette stratégie nécessite que $x$ soit suffisement régulière afin que deux pas de temps "optimaux" successifs ne soient pas trop différents.
	
Une seconde stratégie consiste à prendre un pas de temps (choisi à l'avance ou égal au précedent) et à évaluer l'erreur que l'on commet. Grâce à l'évaluation de l'erreur on déduit un nouveau pas de temps puis refait le calcul de $x^{j+1}$ avec le nouveau pas. Cette stratégie nécessite donc de calculer chaque point deux fois mais semble plus robuste car ne necessite moins d'hypothèses.
	
Ces deux méthodes nécessitent d'évaluer $e^{j+1}$. Cela est possible avec la formule montrée précédemment. On donne cependant une autre expression plus simple de l'erreur facilement transposable dans un algorithme.
Pour cela on calcule $x^{j+1}$ avec la méthode numérique puis on calcule un $x_{new}^j$ en revenant sur nos pas avec la méthode numérique toujours. On a alors :
	
$$x_{new}^j=x^{j+1}-\Delta t f(t_{j+1},x^{j+1})$$
	
donc
	
$$\begin{eqnarray*}
||x_{new}^j-x^j||& = & ||x^{j+1}-\Delta t f(t_{j+1},x^{j+1})-x^j|| \\
& = & ||x^j + \Delta t f(t_j,x^j)-\Delta t f(t_{j+1},x^{j+1})-x^j|| \\
& = & \Delta t ||f(t_{j+1},x^{j+1})-f(t_j,x^j)||
\end{eqnarray*}$$

On déduit donc que 
$$||e^{j+1}||= ||\frac{x_{new}^j-x^j}{2}|| +O(\Delta t^3)$$
		
Cette dernière formule nous donne une évaluation de l'erreur. A partir de l'erreur et de la tolérance souhaitée on calcule le nouveau pas de temps. Pour cela on sait que $e^{j+1}=O(\Delta t^2)$ car :
$$e^{j+1}= \frac{\Delta t^2}{2} [\partial_t{f(t_j,x(t_j))}+\partial_x{f(t_j,x(t_j))}f(t_j,x(t_j))] + O(\Delta t^3)$$
donc :
$$e^{j+1)}=O(\Delta t^2)$$
On comprends alors qu'une diminution quadratique de $\Delta t$ mène à une diminution au première ordre de $e^{j+1}$. En effet $\frac{e^{j+1}}{\Delta t^2}$ est constant dans une première approximation, on obtient donc :
$$\frac{||e^{j+1}||}{\Delta t^2}=\frac{Tol_abs}{\Delta t_{new}^2}$$
On déduit alors que l'on peut prendre un nouveau pas
	
$$\Delta t_{new}=\Delta t \sqrt{\frac{Tol_{abs}}{||e^{j+1}||}}$$	

La fonction ci-dessous met en application la première stratègie évoquée ci-dessus.


In [ ]:
def solve_ivp_euler_explicit_variable_step(f, t0, x0, t_f, dtmin = 1e-16, dtmax = 0.01, atol = 1e-6):
    dt = dtmax/10; # initial integration step
    ts, xs = [t0], [x0]  # storage variables
    t = t0
    ti = 0  # internal time keeping track of time since latest storage point : must remain below dtmax
    x = x0
    while ts[-1] < t_f:
        while ti < dtmax: # on ne veut pas que le temps entre deux enregistrements dépasse dtmax
            t_next, ti_next, x_next = t + dt, ti + dt, x + dt * f(x) # on avance dans le temps de dt, on modifie t, ti et xi en conséquence
            x_back = x_next - dt * f(x_next) # on calcule le x précédent en appliquant Euler à x_next : le résultat est légèrement différent de x (voir dessin)
            ratio_abs_error = atol / (np.linalg.norm(x_back-x)/2) # on calcule l'erreur commise
            dt = 0.9 * dt * np.sqrt(ratio_abs_error) # on calcule le nouveau pas de temps avec la formule calculée précédement (le 0.9 représente la marge de sécurité)
            if dt < dtmin: # si le pas de temps est trop faible, les erreurs d'arrondi vont fausser le résultat : on arrête la simulation
                raise ValueError("Time step below minimum")
            elif dt > dtmax/2: # on empèche le pas de temps d'être trop grand (l'ensemble des calculs précédents ont été faits en supposant dt "petit"...)
                dt = dtmax/2
            t, ti, x = t_next, ti_next, x_next
        dt2DT = dtmax - ti # time left to dtmax ; dt2DT est négatif (à cause de la condition de sortie de boucle)
        t_next, ti_next, x_next = t + dt2DT, 0, x + dt2DT * f(x) # on se replace au temps dtmax, on modifie x en conséquence et on réinitialise la valeur de ti
        ts = np.vstack([ts,t_next])
        xs = np.vstack([xs,x_next])
        t, ti, x = t_next, ti_next, x_next
    return (ts, xs.T)

La fonction solve_ivp_euler_explicit_variable_step permet d'appliquer la méthode d'Euler, en optimisant le pas de temps à chaque étape, afin que l'erreur commise à chaque pas soit proche d'un certain seuil $atol$. Afin que les résultats soient facilement exploitables et plus légers, l'algorithme ne renvoie les valeurs de la fonction qu'avec une période constante $dt_{max}$ que l'on peut voir comme le temps d'échantillonnage de la fonction.

In [ ]:
omega=1
def f(xy):
    x, y = xy
    return np.array([y, -omega*x])

t0, tf, x0 = 0.0, 30.0, np.array([-1.0, 0.0])

t, x = solve_ivp_euler_explicit_variable_step(f, t0, x0, tf)

X=[-np.cos(i) for i in t]
Y=[np.sin(i) for i in t]

plt.figure()
plt.plot(t, x[0],'r') # solution approchée en rouge
plt.plot(t, x[1],'r')
plt.plot(t,X,'g') # solution réelle en vert
plt.plot(t,Y,'g')
plt.grid(True)
plt.figure()
plt.plot(x[0],x[1],'r')
plt.plot(X,Y,'g') # tracé du cercle de centre à l'origine et de rayon 1
plt.plot([0],[0],'.')
plt.axis('equal')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

Lorsque l'on utilise cette fonction pour le système $x'(t)=y(t)$ et $y'(t)=-x(t)$ (de solutions réelles $x(t)=-cos(t)$ et $y(t)=sin(t)$), on remarque que la solution approchée tend à diverger. La quantitée $x^2+y^2$ n'est pas conservée. On retrouve donc le même défaut que la méthode d'Euler classique.

In [ ]:
def f(xabc):
    xa, xb, xc = xabc
    return np.array([-0.04 * xb + 1e4 * xb * xc, 0.04 * xa - 1e4 * xb * xc - 3e7 * xb**2,  3e7 * xb**2])

t0, tf, x0 = 0.0, 5.0, np.array([1.0, 0.0, 0.0])

t, x = solve_ivp_euler_explicit_variable_step(f, t0, x0, tf, dtmin = 1e-20, atol = 1)

L'exemple précédent montre que cet algorithme ne fonctionne pas lorsque la solution prend des valeurs trop grandes et varie trop rapidement. En effet, d'importantes variations augmentent l'erreur commise et requièrent donc l'utilisation d'un pas de temps plus faible. La limite $dt_{min}$ risque alors d'être atteinte. Cet algorithme n'assure alors plus de précision suffisante à cause des erreurs d'arrondi.